https://colab.research.google.com/drive/1QtFPDcvGLRY1h-XjWp43HRahTJzzb35I#scrollTo=PZdyt46KavJ-

In [32]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import shutil

In [33]:
DATASET_ZIP = "processed_casiaweb.zip"
EXTRACT_DIR = "casiaweb_extracted"
MODEL_SAVE_DIR = "face_recognition_model"
MODEL_ZIP = "face_recognition_model.zip"
IMG_SIZE = 112
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 25
TRAIN_SPLIT = 0.8
RANDOM_STATE = 42

In [35]:
print("Extraindo dataset...")
with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

Extraindo dataset...


In [36]:
classes = sorted([d for d in os.listdir(EXTRACT_DIR) if os.path.isdir(os.path.join(EXTRACT_DIR, d))])
print(f"Número de classes: {len(classes)}")
print(f"Classes: {classes}")

Número de classes: 25
Classes: ['1317', '1321', '138', '1542', '1994', '2188', '2189', '2749', '3024', '3384', '3410', '37', '41', '4111', '4670', '4968', '5287', '6090', '6224', '6332', '6374', '658', '819', '872', '956']


In [37]:
# Preparar pastas para treino e validação
train_dir = os.path.join(EXTRACT_DIR, 'train')
val_dir = os.path.join(EXTRACT_DIR, 'val')

# Criar pastas de treino e validação se não existirem
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


In [38]:
import random
for class_name in classes:
    class_dir = os.path.join(EXTRACT_DIR, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)

    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    # Listar todas as imagens da classe
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random.seed(RANDOM_STATE)
    random.shuffle(images)

    # Calcular ponto de divisão
    split_idx = int(len(images) * TRAIN_SPLIT)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    # Copiar para as pastas correspondentes
    for img in train_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(train_class_dir, img)
        shutil.copy(src, dst)

    for img in val_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(val_class_dir, img)
        shutil.copy(src, dst)

    print(f"Classe {class_name}: {len(train_images)} imagens para treino, {len(val_images)} para validação")

Classe 1317: 312 imagens para treino, 79 para validação
Classe 1321: 490 imagens para treino, 123 para validação
Classe 138: 429 imagens para treino, 108 para validação
Classe 1542: 412 imagens para treino, 104 para validação
Classe 1994: 323 imagens para treino, 81 para validação
Classe 2188: 356 imagens para treino, 89 para validação
Classe 2189: 311 imagens para treino, 78 para validação
Classe 2749: 413 imagens para treino, 104 para validação
Classe 3024: 432 imagens para treino, 108 para validação
Classe 3384: 442 imagens para treino, 111 para validação
Classe 3410: 542 imagens para treino, 136 para validação
Classe 37: 384 imagens para treino, 97 para validação
Classe 41: 392 imagens para treino, 98 para validação
Classe 4111: 416 imagens para treino, 105 para validação
Classe 4670: 412 imagens para treino, 103 para validação
Classe 4968: 398 imagens para treino, 100 para validação
Classe 5287: 412 imagens para treino, 104 para validação
Classe 6090: 420 imagens para treino, 105 

In [39]:
# Data augmentation e geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 9889 images belonging to 25 classes.
Found 2484 images belonging to 25 classes.


In [45]:
# Criar modelo MobileNet com batch dimension especificada
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES, batch_size=1):
    # Usar Input com batch_size=1 para garantir que o modelo tenha essa dimensão fixa
    inputs = Input(shape=input_shape, batch_size=batch_size)

    # Base MobileNetV2 sem camadas superiores
    base_model = MobileNetV2(
        input_tensor=inputs,
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    # Congelar as camadas da base MobileNet
    for layer in base_model.layers:
        layer.trainable = False

    # Adicionar as camadas superiores
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Criar o modelo
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [46]:
model = create_model()

<ipython-input-45-531443b24cb5>:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


In [47]:
# Compilar o modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumo do modelo
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (1, 112, 112, 3)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (1, 56, 56, 32)   │        864 │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (1, 56, 56, 32)   │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (1, 56, 56, 32)   │          0 │ bn_Conv1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (1, 56, 56, 32)   │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (1, 56, 56, 32)   │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (1, 56, 56, 32)   │          0 │ expanded_conv_de… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (1, 56, 56, 16)   │        512 │ expanded_conv_de… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (1, 56, 56, 16)   │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (1, 56, 56, 96)   │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (1, 56, 56, 96)   │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (1, 56, 56, 96)   │          0 │ block_1_expand_B… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (1, 57, 57, 96)   │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (1, 28, 28, 96)   │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (1, 28, 28, 96)   │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (1, 28, 28, 96)   │          0 │ block_1_depthwis… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (1, 28, 28, 24)   │      2,304 │ block_1_depthwis… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,595,353 (13.72 MB)

 Trainable params: 1,337,369 (5.10 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [48]:
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [49]:
# Treinar o modelo com lotes normais para eficiência
model_for_training = create_model(batch_size=None)
model_for_training.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_for_training.summary()

<ipython-input-45-531443b24cb5>:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 56, 56,    │        864 │ input_layer_11[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 56, 56,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 56, 56,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 56, 56,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 56, 56,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 56, 56,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 56, 56,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 56, 56,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 57, 57,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 28, 28,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 28, 28,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 28, 28,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 28, 28,    │      2,304 │ block_1_depthwis

 Total params: 3,595,353 (13.72 MB)

 Trainable params: 1,337,369 (5.10 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [50]:
# Treinar o modelo
history = model_for_training.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.2177 - loss: 2.9347
Epoch 1: val_accuracy improved from -inf to 0.40300, saving model to best_model.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 180s 562ms/step - accuracy: 0.2179 - loss: 2.9331 - val_accuracy: 0.4030 - val_loss: 1.9473
Epoch 2/10
  1/309 ━━━━━━━━━━━━━━━━━━━━ 1:28 288ms/step - accuracy: 0.3750 - loss: 2.1460

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.40300 to 0.41599, saving model to best_model.h5


...

309/309 ━━━━━━━━━━━━━━━━━━━━ 201s 650ms/step - accuracy: 0.4093 - loss: 1.9735 - val_accuracy: 0.4481 - val_loss: 1.8032
Epoch 4/10
  1/309 ━━━━━━━━━━━━━━━━━━━━ 1:29 292ms/step - accuracy: 0.3438 - loss: 1.7743
Epoch 4: val_accuracy did not improve from 0.44805
309/309 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.3438 - loss: 1.7743 - val_accuracy: 0.4472 - val_loss: 1.8115
Epoch 5/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step - accuracy: 0.4535 - loss: 1.8095
Epoch 5: val_accuracy improved from 0.44805 to 0.46631, saving model to best_model.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 170s 549ms/step - accuracy: 0.4535 - loss: 1.8095 - val_accuracy: 0.4663 - val_loss: 1.6974
Epoch 6/10
  1/309 ━━━━━━━━━━━━━━━━━━━━ 1:52 366ms/step - accuracy: 0.4062 - loss: 1.8800
Epoch 6: val_accuracy did not improve from 0.46631
309/309 ━━━━━━━━━━━━━━━━━━━━ 41s 130ms/step - accuracy: 0.4062 - loss: 1.8800 - val_accuracy: 0.4631 - val_loss: 1.7092
Epoch 7/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step - accuracy: 0.4727 - loss: 1.7570
Epoch 7: val_accuracy improved from 0.46631 to 0.49432, saving model to best_model.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 168s 544ms/step - accuracy: 0.4727 - loss: 1.7569 - val_accuracy: 0.4943 - val_loss: 1.6775
Epoch 8/10
  1/309 ━━━━━━━━━━━━━━━━━━━━ 1:28 286ms/step - accuracy: 0.6250 - loss: 1.2537
Epoch 8: val_accuracy improved from 0.49432 to 0.50203, saving model to best_model.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 27s 87ms/step - accuracy: 0.6250 - loss: 1.2537 - val_accuracy: 0.5020 - val_loss: 1.6726
Epoch 9/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.4865 - loss: 1.7134
Epoch 9: val_accuracy improved from 0.50203 to 0.53450, saving model to best_model.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 202s 653ms/step - accuracy: 0.4865 - loss: 1.7133 - val_accuracy: 0.5345 - val_loss: 1.5529
Epoch 10/10
  1/309 ━━━━━━━━━━━━━━━━━━━━ 1:42 334ms/step - accuracy: 0.6250 - loss: 1.0924
Epoch 10: val_accuracy did not improve from 0.53450
309/309 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.6250 - loss: 1.0924 - val_accuracy: 0.5276 - val_loss: 1.5908
Restoring model weights from the end of the best epoch: 9.


In [51]:
# Transferir pesos do modelo treinado para o modelo com batch size fixo
model.set_weights(model_for_training.get_weights())

In [52]:
plt.figure(figsize=(12, 4))

# Gráfico de acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Gráfico de perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves.png')
plt.close()

In [53]:
# Avaliar o modelo
evaluation = model.evaluate(
    val_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=1,  # Usar batch size 1 para avaliação
        class_mode='categorical',
        shuffle=False
    ),
    steps=len(os.listdir(val_dir))
)

print(f"Avaliação no conjunto de validação: Perda = {evaluation[0]}, Acurácia = {evaluation[1]}")

Found 2484 images belonging to 25 classes.
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.5681 - loss: 1.4473
Avaliação no conjunto de validação: Perda = 2.014310359954834, Acurácia = 0.4399999976158142


In [54]:
def predict_image(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão de lote

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)

    return predicted_class, confidence, prediction

In [55]:
# Testar algumas imagens aleatórias
test_images = []
for class_name in os.listdir(val_dir):
    class_path = os.path.join(val_dir, class_name)
    if os.path.isdir(class_path):
        images = [os.path.join(class_path, img) for img in os.listdir(class_path)
                 if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if images:
            test_images.append(random.choice(images))
            if len(test_images) >= 5:  # Limitar a 5 imagens de teste
                break

# Classe para mapear índices para nomes de classe
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}

# Fazer predições
print("\nResultados de Inferência:")
for image_path in test_images:
    true_class = os.path.basename(os.path.dirname(image_path))
    predicted_class_idx, confidence, predictions = predict_image(image_path, model)
    predicted_class = class_names[predicted_class_idx]

    print(f"Imagem: {os.path.basename(image_path)}")
    print(f"Classe verdadeira: {true_class}")
    print(f"Classe prevista: {predicted_class} (índice {predicted_class_idx})")
    print(f"Confiança: {confidence:.4f}")
    print(f"Top 3 predições: {sorted(zip(class_names.values(), predictions[0]), key=lambda x: x[1], reverse=True)[:3]}")
    print("-" * 50)


Resultados de Inferência:
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Imagem: 293871.jpg
Classe verdadeira: 5287
Classe prevista: 5287 (índice 16)
Confiança: 0.2103
Top 3 predições: [('5287', np.float32(0.21031173)), ('1321', np.float32(0.14677791)), ('3384', np.float32(0.12132401))]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Imagem: 207841.jpg
Classe verdadeira: 3410
Classe prevista: 6332 (índice 19)
Confiança: 0.4192
Top 3 predições: [('6332', np.float32(0.41919267)), ('3410', np.float32(0.18486783)), ('1321', np.float32(0.11359648))]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Imagem: 107394.jpg
Classe verdadeira: 1321
Classe prevista: 3384 (índice 9)
Confiança: 0.2765
Top 3 predições: [('3384', np.float32(0.27646044)), ('3024', np.float32(0.22786985)), ('1321', np.float32(0.19808176))]
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Imagem: 68438.jpg
Classe 

In [56]:
print("Salvando o modelo...")
if os.path.exists(MODEL_SAVE_DIR):
    shutil.rmtree(MODEL_SAVE_DIR)
os.makedirs(MODEL_SAVE_DIR)

# Verificar as formas dos tensores de entrada e saída
print(f"Forma do tensor de entrada: {model.input.shape}")
print(f"Forma do tensor de saída: {model.output.shape}")

Salvando o modelo...
Forma do tensor de entrada: (1, 112, 112, 3)
Forma do tensor de saída: (1, 25)


In [57]:
# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, MODEL_SAVE_DIR)

In [58]:
# Comprimir o modelo salvo em um arquivo zip
print(f"Comprimindo o modelo em {MODEL_ZIP}...")
with zipfile.ZipFile(MODEL_ZIP, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(MODEL_SAVE_DIR):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, os.path.dirname(MODEL_SAVE_DIR)))

print(f"Modelo salvo e comprimido com sucesso em {MODEL_ZIP}")

Comprimindo o modelo em face_recognition_model.zip...
Modelo salvo e comprimido com sucesso em face_recognition_model.zip


In [59]:
# Verificar se o modelo tem a assinatura correta
saved_model = tf.saved_model.load(MODEL_SAVE_DIR)
print("\nInformações da assinatura do modelo salvo:")
print(saved_model.signatures["serving_default"])

# Limpar diretórios temporários se necessário
# shutil.rmtree(EXTRACT_DIR)
print("Processo concluído!")


Informações da assinatura do modelo salvo:
ConcreteFunction Input Parameters:
  inputs (KEYWORD_ONLY): TensorSpec(shape=(1, 112, 112, 3), dtype=tf.float32, name='inputs')
Output Type:
  Dict[['output_0', TensorSpec(shape=(1, 25), dtype=tf.float32, name='output_0')]]
Captures:
  135411205697808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205696272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205696080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205696656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205696464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205695120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205693392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205693200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205693776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205693584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135411205692240: Ten